In [1]:
import requests 
import pandas as pd
from requests import exceptions
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import argparse
import os
import sys
import psycopg2

/Applications/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
EXCEPTIONS = set([IOError, FileNotFoundError, exceptions.RequestException, exceptions.HTTPError,
                 exceptions.ConnectionError, exceptions.Timeout])

In [3]:
class database:
    def __init__(self, dbname, user, password):
        host = "'localhost'"
        port="'5432'"
        db = "dbname =  '"+ str(dbname) + "' user= '"+str(user)+r"' password= '"+str(password) + "' host = "+str(host)+" port = " + str(port)
        db = str(db)
        print (db)
        #conn = psycopg2.connect("dbname = 'web_scrapper_db' user='postgres' password='postgresql123' host = 'localhost' port='5432'")
        conn = psycopg2.connect(db)
        cur = conn.cursor()
        cur.execute("CREATE TABLE IF NOT EXISTS real_madrid (Name_of_video text, search_criteria text, Date_added text, Duration text, cast_name text, category text, description text, Tags text, Fembed_Link text)")
        conn.commit()
        conn.close()

    def insert(Name_of_video,Date_added, Duration,cast_name,Category, Description, Tags, search_criteria, Fembed_Link):
        conn = psycopg2.connect("dbname = 'web_scrapper_db' user='postgres' password='postgres123' host = 'localhost'port='5432'")
        cur = conn.cursor() 
        cur.execute("INSERT INTO real_madrid VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",(Name_of_video,Date_added, Duration,cast_name,Category, Description, Tags, search_criteria, Fembed_Link))
        conn.commit()
        conn.close()

    def view():
        conn = psycopg2.connect("dbname = 'web_scrapper_db' user='postgres' password='postgres123' host = 'localhost'port='5432'")
        cur = conn.cursor()
        cur.execute("SELECT * FROM real_madrid")
        view = cur.fetchall()
        conn.close()
        return view
database('web_scrapper_db' , 'postgres', 'postgres123') #pass your credentials in strings like this

dbname =  'web_scrapper_db' user= 'postgres' password= 'postgres123' host = 'localhost' port = '5432'


In [4]:
def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                #f.flush() commented by recommendation from J.F.Sebastian
    return local_filename

===============================================JAVqd.COM==============================================================

In [29]:
search_criteria = "beautigul-girl" #put '-' instead of space
url = "https://www.javqd.com/search/movie/"+ search_criteria +"/page-1.html"
print (url)
web= requests.get(url)
content = web.content
full_search_return = soup(content, "html.parser")
video_content_list = full_search_return.find_all("div", {"class":"col-lg-3 col-md-4 col-sm-6 videoitem"})
total_num_dis_page = 32
total_number_result_page = full_search_return.find("div", {"class": "col-xs-12 border"}).text.split(' ')[1]
total_number_result_page = int (int(total_number_result_page)/total_num_dis_page)
total_number_result_page
if total_number_result_page ==  0:
    total_number_result_page = 1
print(total_number_result_page)

https://www.javqd.com/search/movie/beautigul-girl/page-1.html
81


In [35]:
for page in range (total_number_result_page):
    if page > 12:
        url = "https://www.javqd.com/search/movie/"+ search_criteria +"/page-" + str(page)+".html"
        web= requests.get(url)
        content = web.content
        full_search_return = soup(content, "html.parser")
        video_content_list = full_search_return.find_all("div", {"class":"col-lg-3 col-md-4 col-sm-6 videoitem"})
        total_num_dis_page = 32
        total_number_result_page = full_search_return.find("div", {"class": "col-xs-12 border"}).text.split(' ')[1]
        total_number_result_page = int (int(total_number_result_page)/total_num_dis_page)
        total_number_result_page
        try:
            for video in range (total_num_dis_page):
                link_video_main_page = str(video_content_list[video].find("a")).split('"')[1]
                link_video_main_page = "https://www.javqd.com" + link_video_main_page
                #print (link_video_main_page)
                web_main_page = requests.get(link_video_main_page)
                content_main_page = web_main_page.content
                content_parsed = soup(content_main_page, "html.parser")
                link = str(content_parsed.find("textarea" ,{"class":"select-all"})).split('"')[13]
                #print (link)
                browser = webdriver.Chrome(r'/Users/MacBookPro/Desktop/Real_Madrid/chromedriver')
                wait = WebDriverWait(browser, 10)
                browser.get(link)
                elements = browser.find_element_by_css_selector("#external-embed").get_attribute("src")
                fembed_link = elements
                print (fembed_link)
                browser.close()
                Name_of_video = content_parsed.find("h2", {"class":"info-box-heading"}).text
                #print(Name_of_video)
                v_info = content_parsed.find("div", {"class": "tab-content tab-description active"}).text.lower().split("\n")

                try:
                    for item in v_info:
                        if item == "added:" or item == "added":
                            index = v_info.index(item)
                            date_added = v_info[int(index) + 1]
                            #print (date_added)
                except Exception as e:
                        date_added = "NaN"

                try:
                    for item in v_info:
                        if item == "duration:" or item == "duration":
                            index = v_info.index(item)
                            duration = v_info[int(index) + 1]
                            #print (duration)
                except Exception as e:
                        duration = "NaN"

                try:
                    for item in v_info:
                        if item == "cast:" or item == "cast":
                            index = v_info.index(item)
                            cast_name = v_info[int(index) + 1]
                            #print (cast_name)
                except Exception as e:
                        cast_name = "NaN"

                try:
                    for item in v_info:
                        if item == "category:" or item == "category":
                            index = v_info.index(item)
                            category = v_info[int(index) + 1]
                            #print (category)
                except Exception as e:
                        category = "NaN"

                try:
                    for item in v_info:
                        if item == "description:" or item == "description":
                            index = v_info.index(item)
                            description = str(v_info[int(index) + 1])
                            #print (description)
                except Exception as e:
                        description = "NaN"

                try:
                    for item in v_info:
                        if item == "tags:" or item == "tags":
                            index = v_info.index(item)
                            tags = v_info[int(index) + 1: len(v_info)]
                            #print (tags)
                except Exception as e:
                        Tags = "NaN"


                database.insert(Name_of_video , search_criteria , date_added , duration , cast_name , category , description , tags , fembed_link)
        except Exception as e:
            continue
    else:
        page = page + 1
        print(page)

1
2
3
4
5
6
7
8
9
10
11
12
13
https://www.fembed.com/v/40oxn-lyyv8
https://www.fembed.com/v/1xoqd0-exv4
https://www.fembed.com/v/6mo2nyn1dor
https://www.fembed.com/v/5jv4e5kexo0
https://www.fembed.com/v/dw9r3-eqp9g
https://www.fembed.com/v/yxv3lx0mqvl
https://www.fembed.com/v/05ole2pznv6
https://www.fembed.com/v/env8k66l0v1
https://www.fembed.com/v/8goz1lgkyvd
https://www.fembed.com/v/8xop7dngqo7
https://www.fembed.com/v/1l96k3n5lo5
https://www.fembed.com/v/80oe16427vj
https://www.fembed.com/v/80oe1rlq7vj
https://www.fembed.com/v/40oxnlw8yv8
https://www.fembed.com/v/dwor3w76pog
https://www.fembed.com/v/7zv--m2r2vx
https://www.fembed.com/v/809e1rjm7oj
https://www.fembed.com/v/6mo2n7n4dor
https://www.fembed.com/v/8gvz1l58y9d
https://www.fembed.com/v/1xvqdwgxx94
https://www.fembed.com/v/dwor3wpkpog
https://www.fembed.com/v/1lo6kplqlv5
https://www.fembed.com/v/7yvwzyn0xvj
https://www.fembed.com/v/8g9z12g6yod
https://www.fembed.com/v/8gody7ynpoy
https://www.fembed.com/v/dwor31w0pog
https://

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=67.0.3396.99)
  (Driver info: chromedriver=2.40.565386 (45a059dc425e08165f9a10324bd1380cc13ca363),platform=Mac OS X 10.13.5 x86_64)


In [23]:
print(page)

43


In [ ]:
link_video_main_page = str(video_content_list[videos].find("a")).split('"')[1]
link_video_main_page = "https://www.javqd.com" + link_video_main_page
link_video_main_page

In [ ]:
web_main_page = requests.get(link_video_main_page)
content_main_page = web_main_page.content
content_parsed = soup(content_main_page, "html.parser")
link = str(content_parsed.find("textarea" ,{"class":"select-all"})).split('"')[13]
link

In [ ]:
browser = webdriver.Chrome(r'/Users/MacBookPro/Desktop/Real_Madrid/chromedriver')
wait = WebDriverWait(browser, 10)
browser.get(link)
elements = browser.find_element_by_css_selector("#external-embed").get_attribute("src")
fembed_link = elements
print (fembed_link)
browser.close()

In [ ]:
###final link needed for downloading
browser = webdriver.Chrome(r'/Users/MacBookPro/Desktop/Real_Madrid/chromedriver')
wait = WebDriverWait(browser, 10)
browser.get(elements)
elements = browser.find_element_by_css_selector("video").get_attribute("src")
print (elements)
browser.close()

In [ ]:
download_file(elements)

===========================================West(yourporn.sexy)=================================================

#get full website
web = requests.get("https://yourporn.sexy/sexy-CHINESE-GIRL.html")
content = web.content
full_content = soup(content, "html.parser")
content_1 = full_content.find_all("div", {"class" : "post_el_small"})

#get prelink
view_1 = content_1[0].find_all("a")
prelink = ""
count= 0
for links in view_1:
    if count == 0:
        string = str(links)
        if "post" in string:
            prelink = string
            #print(prelink)
            count +=1

#https://s12-1.trafficdeposit.com/bvideo/OveI6-OWRtcnl2JyZx29Fg/1531539661/5a0b5ff2d94da/5a84e8b3790a5.mp4

#get link for video
prelink = prelink.split()
link = prelink[1]
link = link[6:len(link) - 6]

link = "https://yourporn.sexy"+link
link

web_2 = requests.get("https://yourporn.sexy/post/5a84e8b3790a5.html?sk=sexy-CHINESE-GIRL&amp;so=0&amp;ss=latest")
content_2 = web_2.content
full_content_1 = soup(content_2, "html.parser")

#get all the sub_directiorys

"https://s12-1.trafficdeposit.com/bvideo/uCPg0lLvdc4mRUHDttbb-Q/1531585511/5a0b5ff2d94da/5a84e8b3790a5.mp4"
"https://con1.trafficdeposit.com/bvideo/con2/con3/con4/con5.mp4"

token = full_content_1.find_all("span",  {"class":"vidsnfo"})
con1 = str(token).split("/")[2][:-1]
con2 = str(token).split("/")[3][:-1]
con3 = str(token).split("/")[4][:-1]
con5 = str(token).split('"')[3]
token_1 = full_content_1.find_all("div", {"class":"pes_author_div pes_edit_div transition"})
con4 = str(token_1).split('"')[3]
print(con1, con2, con3, con4, con5)
final_link = "https://"+con1+".trafficdeposit.com/bvideo/"+con2+"/"+con3+"/"+con4+"/"+con5+".mp4"
print (final_link)

download_file(final_link)